# Test 'bedrock' and 'bedrock-runtime' Python SDKs 

# MISSING:
- EXAMPLE FOR TOOL USE
- EXAMPLE FOR ASYNC INVOCATIONS

### Summary

The following actions are supported by Amazon Bedrock Runtime:
- [ ] ApplyGuardrail
- [x] Converse
- [ ] ConverseStream
- [ ] GetAsyncInvoke
- [x] InvokeModel
- [ ] InvokeModelWithResponseStream
- [ ] ListAsyncInvokes
- [ ] StartAsyncInvoke

### List Models

- 'bedrock' client

In [9]:
"""
Lists the available Amazon Bedrock models in an AWS Region.
"""

import boto3
import pandas as pd

bedrock_client = boto3.client(service_name="bedrock", region_name="us-east-1")
response = bedrock_client.list_foundation_models()

models = response["modelSummaries"]

print(f"Got {len(models)} foundation models.\n")
display(pd.DataFrame(models).head())

Got 80 foundation models.



,modelArn,modelId,modelName,providerName,inputModalities,outputModalities,responseStreamingSupported,customizationsSupported,inferenceTypesSupported,modelLifecycle
0,arn:aws:bedrock:us-east-1::foundation-model/am...,amazon.titan-tg1-large,Titan Text Large,Amazon,[TEXT],[TEXT],True,[],[ON_DEMAND],{'status': 'ACTIVE'}
1,arn:aws:bedrock:us-east-1::foundation-model/am...,amazon.titan-image-generator-v1:0,Titan Image Generator G1,Amazon,"[TEXT, IMAGE]",[IMAGE],NaN,[FINE_TUNING],[PROVISIONED],{'status': 'ACTIVE'}
2,arn:aws:bedrock:us-east-1::foundation-model/am...,amazon.titan-image-generator-v1,Titan Image Generator G1,Amazon,"[TEXT, IMAGE]",[IMAGE],NaN,[],[ON_DEMAND],{'status': 'ACTIVE'}
3,arn:aws:bedrock:us-east-1::foundation-model/am...,amazon.titan-image-generator-v2:0,Titan Image Generator G1 v2,Amazon,"[TEXT, IMAGE]",[IMAGE],NaN,[FINE_TUNING],"[PROVISIONED, ON_DEMAND]",{'status': 'ACTIVE'}
4,arn:aws:bedrock:us-east-1::foundation-model/am...,amazon.titan-text-premier-v1:0,Titan Text G1 - Premier,Amazon,[TEXT],[TEXT],True,[],[ON_DEMAND],{'status': 'ACTIVE'}


### Invoke Model

- `invoke_model` method
- 'bedrock-runtime' client
- Single message as input and output.
- Uses the native inference api of each model

In [4]:
# Use the native inference API to send a text message to Amazon Titan Text G1 - Express.

import boto3
import json

from botocore.exceptions import ClientError

# Create an Amazon Bedrock Runtime client.
brt = boto3.client("bedrock-runtime")

# Set the model ID, e.g., Amazon Titan Text G1 - Express.
model_id = "amazon.titan-text-express-v1"

# Define the prompt for the model.
prompt = "Describe the purpose of a 'hello world' program in one line."

# Format the request payload using the model's native structure.
native_request = {
    "inputText": prompt,
    "textGenerationConfig": {
        "maxTokenCount": 512,
        "temperature": 0.5,
        "topP": 0.9
    },
}

# Convert the native request to JSON.
request = json.dumps(native_request)

try:
    # Invoke the model with the request.
    response = brt.invoke_model(modelId=model_id, body=request)

except (ClientError, Exception) as e:
    print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
    exit(1)

# Decode the response body.
model_response = json.loads(response["body"].read())

# Extract and print the response text.
response_text = model_response["results"][0]["outputText"]
print(response_text)


A 'hello world' program is a basic computer program that outputs the message "hello world" to the console.


---
### Converse

- `converse` method
- 'bedrock-runtime' client
- Allows multiple messages as input, outputs a single message.
- Unified inference api across models.

In [5]:
# Use the Conversation API to send a text message to Amazon Titan Text G1 - Express.

import boto3
from botocore.exceptions import ClientError

# Create an Amazon Bedrock Runtime client.
brt = boto3.client("bedrock-runtime")

# Set the model ID, e.g., Amazon Titan Text G1 - Express.
model_id = "amazon.titan-text-express-v1"

# Start a conversation with the user message.
user_message = "Describe the purpose of a 'hello world' program in one line."
conversation = [
    {
        "role": "user",
        "content": [{"text": user_message}],
    }
]

try:
    # Send the message to the model, using a basic inference configuration.
    response = brt.converse(
        modelId=model_id,
        messages=conversation,
        inferenceConfig={"maxTokens": 512, "temperature": 0.5, "topP": 0.9},
    )

    # Extract and print the response text.
    response_text = response["output"]["message"]["content"][0]["text"]
    print(response_text)

except (ClientError, Exception) as e:
    print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
    exit(1)



The purpose of a 'hello world' program is to demonstrate the basic functionality of a programming language or environment by printing the phrase 'Hello, World!' to the console.
